In [2]:
import json
import pandas as pd

In [16]:
partidos_votos = json.load(open("data/partidos_votos_total.json"))


In [17]:
def json_to_df(votos, column_name):
    all_props = []
    for row in votos:
        party_name = row["nome"]
        party_id = row["id"]
        propositions = row['temas']
        for prop in propositions:
            dict_prop = {column_name:party_name}
            dict_prop.update(prop)
            all_props.append(dict_prop)
    all_props = pd.DataFrame(all_props)
    return all_props

In [18]:
all_props = json_to_df(partidos_votos, "party_name")
props_by_party = all_props[["party_name", "tema", "value"]].pivot(index="party_name", columns='tema', values='value').to_dict('index')

KeyError: 'id'

In [122]:
no_vote = "-10"
def calc_sim(propsA, propsB):
    equals = 0
    to_sum = 0
    for prop, vA in propsA.iteritems():
        if vA == no_vote:
            continue
        vB = propsB[prop]
        if vB == no_vote:
            continue
        if vA == vB:
            equals +=1
        to_sum+=1
    if to_sum == 0:
        return None, to_sum
    else:
        return equals / float(to_sum), to_sum
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": 0}) == (1, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": 1}) == (.5, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 0, "b": 1}) == (0.0, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": no_vote}) == (1.0, 1)
assert calc_sim({"a": 1, "b": 0}, {"a": no_vote, "b": no_vote}) == (None, 0)

In [123]:
all_sim = []
parties_already_calculates = set()

def get_key(partyA, partyB):
    return "{}_{}".format(partyA, partyB)

for partyA, propsA in props_by_party.iteritems():
    for partyB, propsB in props_by_party.iteritems():
        if get_key(partyA, partyB) in parties_already_calculates or \
            get_key(partyB, partyA) in parties_already_calculates:
            continue

        if partyA == partyB:
            sim, used = 1,1
        else:
            sim, used = calc_sim(propsA, propsB)
        
        all_sim.append({"source": partyA, "target": partyB, "value": sim, "used": used})
        if partyA != partyB:
            all_sim.append({"target": partyA, "source": partyB, "value": sim, "used": used})
        parties_already_calculates.add(get_key(partyA, partyB))
        parties_already_calculates.add(get_key(partyB, partyA))

In [130]:
all_sim_df[all_sim_df.source == "pcb"]

,source,target,used,value


In [127]:
all_sim_df = pd.DataFrame(all_sim)
all_sim_df[all_sim_df.source == "pros"].sort_values(by="value", ascending=False)

,source,target,used,value
700,pros,pros,1,1.000000
725,pros,avante,1,1.000000
148,pros,prb,24,0.666667
713,pros,psl,24,0.666667
711,pros,ptdob,23,0.652174
364,pros,prp,20,0.650000
28,pros,ptn,22,0.636364
205,pros,ptc,19,0.631579
460,pros,pmn,19,0.631579
707,pros,psdc,19,0.631579


In [100]:
 set(all_sim_df[(all_sim_df.used >= 5)].source.drop_duplicates()) - set(all_sim_df[(all_sim_df.used >= 5) & (all_sim_df.value >= 0.75)].source.drop_duplicates())

{u'pps', u'pros', u'psb', u'pv'}

In [101]:
set(all_sim_df.source.drop_duplicates()) - set(all_sim_df[(all_sim_df.used >= 5)].source.drop_duplicates())

{u'avante', u'pmb', u'podemos'}

In [108]:
temp = all_sim_df[(all_sim_df.used >= 5) & (all_sim_df.value >= 0.75)]
all_sim = temp.to_dict('records')
parties = temp.source.drop_duplicates()
parties_without_similarity =  set(all_sim_df[(all_sim_df.used >= 5)].source.drop_duplicates()) - \
set(all_sim_df[(all_sim_df.used >= 5) & (all_sim_df.value >= 0.75)].source.drop_duplicates())
parties = list(parties) + list(parties_without_similarity)

In [109]:
len(all_sim)

158

In [110]:
for i, s in enumerate(all_sim):
    all_sim[i]['source'] = nodes_name_to_index[all_sim[i]['source']]
    all_sim[i]['target'] = nodes_name_to_index[all_sim[i]['target']]

In [114]:
def get_group(party):
    if party in ["psol", "pt", "pdt", "pcdob", "rede"]:
        return "red"
    elif party in ["dem", "solidaried", "psdb"]:
        return "blue"
    elif party in ["ptn", "prp", "pmn", "prtb", "ptdob", "ptc", "psdc"]:
        return "green"
    elif party in ["prb", "psl", "pp", "ptb", "pmdb", "psd", "pen", "pr", "phs", "psc"]:
        return "orange"
    else:
        return "black"
nodes = [{"id": n, "group": get_group(n)} for n in parties]
nodes_name_to_index = {n['id']: i for i,n in enumerate(nodes)}

    
links = all_sim

data = {"nodes": nodes,
        "links": links}

json.dump(data, open("analyses/data.json", "w"))

# Compare parties and deputies

In [3]:
def json_to_df(votos, column_name, id_column):
    all_props = []
    for row in votos:
        party_name = row["nome"]
        party_id = row[id_column]
        propositions = row['temas']
        for prop in propositions:
            dict_prop = {column_name:party_name, "id":party_id}
            dict_prop.update(prop)
            all_props.append(dict_prop)
    all_props = pd.DataFrame(all_props)
    return all_props

partidos_votos = json.load(open("data/partidos_votos_total.json"))
deputados_votos = json.load(open("data/deputados_votos_total.json"))

In [4]:
new_names_parties = {r['old_name']: r['new_name'] for i, r in pd.read_csv("data/parties_new_names.csv").iterrows()}

In [5]:
new_deputies_info = pd.read_csv("data/deputados_info_by_api.csv")

In [6]:
deputies_props = json_to_df(deputados_votos, "deputy_name", "id_dep")
deputies_props["id"] = deputies_props["id"].astype(int)
props_by_deputy = deputies_props[["deputy_name", "tema", "value"]].pivot(index="deputy_name", columns='tema', values='value').to_dict('index')

In [7]:
deputy_party_df = pd.merge(deputies_props, new_deputies_info[["id","nome","siglaPartido"]], on="id", how="left")[["deputy_name", "siglaPartido"]].drop_duplicates(["deputy_name", "siglaPartido"])
deputy_party_df['siglaPartido'] = deputy_party_df['siglaPartido'].apply(lambda x: x.lower())
change = {"pode": "podemos", "sd": "solidaried"}
deputy_party_df.siglaPartido = deputy_party_df.siglaPartido.apply(lambda x: change[x] if x in change else x)
part_by_deputy = {d['deputy_name']: d['siglaPartido'] for index, d in deputy_party_df.iterrows()}
# part_by_deputy = {d['nome']: d['partido'] for d in deputados_votos}

In [8]:
party_props = json_to_df(partidos_votos, "party_name", "nome")
props_by_party = party_props[["party_name", "tema", "value"]].pivot(index="party_name", columns='tema', values='value').to_dict('index')

In [9]:
no_vote_A = "-1"
no_vote_B = "-10"
ABSTENCAO = "-2"
OBSTRUCAO = "-3"
ARTIGO_17 = "-4"
PARTY_LIBERADO = "-1"
def calc_sim(propsA, propsB):
    equals = 0
    to_sum = 0
    for prop, vA in propsA.iteritems():
        vA = str(vA)
        if vA in [no_vote_A, ABSTENCAO, ARTIGO_17]: #SÓ PASSA 0, 1, -3
            continue
        if vA == OBSTRUCAO: #SÓ PASSA 0, 1
            vA = "0"
        vB = str(propsB[prop])
        if vB == no_vote_B: #SÓ PASSA 0, 1, -1
            continue
        if vB == PARTY_LIBERADO:
            equals += 1
            to_sum += 1
        elif vA == vB and vA in ["1", "0"]:
            equals +=1
            to_sum+=1
        else:
            to_sum+=1
            if vA not in ["1", "0"] and vB not in ["1", "0"]:
                print(vA, vB)
    if to_sum == 0:
        return None, to_sum
    else:
        return equals / float(to_sum), to_sum
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": 0}) == (1, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": 1}) == (.5, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 0, "b": 1}) == (0.0, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": no_vote_B}) == (1.0, 1)
assert calc_sim({"a": 1, "b": 0}, {"a": no_vote_B, "b": no_vote_B}) == (None, 0)

In [10]:
all_sim = []
def get_key(partyA, partyB):
    return "{}_{}".format(partyA, partyB)

for deputy, props_deputy in props_by_deputy.iteritems():
    party = part_by_deputy[deputy]
    if party == 's.part.':
        print 's.part.'
        continue
    props_party = props_by_party[party]
    sim, used = calc_sim(props_deputy, props_party)
        
    all_sim.append({"source": deputy, "target": party, "value": sim, "used": used})

s.part.
s.part.
s.part.
s.part.
s.part.


In [14]:
df = pd.DataFrame(all_sim)
df.rename(columns={"source": "deputado", "target": "partido", "used": "votacoes", "value": "similaridade"}, inplace=True)

In [25]:
df['similaridade'] = df['similaridade']*100

In [27]:
json.dump(df[df.similaridade.notnull()].sort_values(by="similaridade", ascending=False).to_dict("records"), open("data/deputado_partido_similaridade.json", "w"))

In [56]:
df[df.used >= 5].value.describe()

count    493.000000
mean       0.866490
std        0.127839
min        0.333333
25%        0.800000
50%        0.900000
75%        0.956522
max        1.000000
Name: value, dtype: float64